In [1]:
# your other imports here ...
import sys, os
import pandas as pd
import matplotlib.pyplot as plt

# TODO: replace with your path/to/ninarow
ninarowdir = os.path.dirname(os.getcwd())
modelfitdir = ninarowdir + "/model_fitting/"
# os.listdir(modelfitdir)

# sets the import path to the model-fitting directory
sys.path.insert(0, modelfitdir)
from parsers import *
# from new_fit import *
from utils import *
from utils import show
from tqdm import tqdm
import random

# WARNING: %load_ext autoreload and %autoreload 2 may interfere with 
# the Multi-threading processes!
%load_ext autoreload
%autoreload 2

In [2]:
data_path = "../data"
output_path = "../data/out"
n_splits = 5
fold_number = 1
threads = 1
random_sample = False
verbose = True

print(f"Building output directory at {output_path}")
os.makedirs(output_path, exist_ok = True)

# first, we have to check to see if all the splits are there ...
assert np.all([f"{i + 1}.csv" in os.listdir(data_path) for i in range(n_splits)])
print("Detected splits in this directory. Loading splits ...")

# then we read them in
splits = [pd.read_csv(f"{data_path}/{i + 1}.csv") for i in range(n_splits)]

# we convert every row of our CSV to a "CSVMove object" using df_to_CSVMove - we do so for all the splits
# CSVMove is a class that is defined in the parsers.py file 
fold_data = [[csvmove for csvmove in df_to_CSVMove(split, warn = False)] for split in splits]

Building output directory at ../data/out
Detected splits in this directory. Loading splits ...


In [32]:
testloader = DataLoader(splits[0])
trainloader = DataLoader(pd.concat([splits[1], splits[2], splits[3], splits[4]]))


In [14]:
splits[2]

,black,white,color,move,response_time,participant_id
0,2105376,4194320,White,8,1,1
1,4294967552,4194304,White,2097152,1,1
2,2101254,56,White,64,1,1
3,4296016128,6291456,White,8388608,1,1
4,6979321856,6291456,White,1048576,1,1
5,4096,0,White,16,1,1


In [ ]:
import model
from time import time
# from utils import *

random.seed(10)
model.initialize_thread_pool(1, manual_seed = 10)
model_fitter =  model.ModelFitter(model.DefaultModel(), threads = threads, verbose = True)

tick = time()
q = model_fitter.fit(splits[2])
print(time() - tick)
q

Setting manual seed 10 for single-thread
Thread 0: Base Seed 10, Seed: 10, Random Number: 601088376405717203

[Preprocessing] Initial log-likelihood estimation


100%|██████████| 10/10 [00:07<00:00,  1.37it/s]


	[0] NLL: 13.4516 Params: [2.001, 0.02, 0.2, 0.05, 1.2, 0.801, 1.001, 0.4, 3.501, 5.0]
Beginning optimization of a STOCHASTIC objective function

 Iteration    f-count      E[f(x)]        SD[f(x)]           MeshScale          Method              Actions
     0           1         13.4516             nan               1                                  
	[1] NLL: 19.5374 Params: [1.053, 0.68, 0.147, 0.292, 0.539, 0.952, -1.899, -1.372, -4.072, -3.799]
	[2] NLL: 17.1197 Params: [1.334, 0.294, 0.414, 0.005, 1.939, -4.663, -4.106, 4.424, -3.281, 2.393]
	[3] NLL: 19.682 Params: [1.632, 0.782, 0.037, 0.156, 1.082, -3.052, 3.237, 1.782, -0.542, -3.545]
	[4] NLL: 19.5025 Params: [1.913, 0.396, 0.272, 0.423, 1.346, 1.294, 0.444, -4.99, -2.114, 0.264]
	[5] NLL: 12.3731 Params: [2.396, 0.624, 0.304, 0.122, 1.769, -2.373, 1.753, -1.235, 2.314, -0.088]
	[6] NLL: 19.1623 Params: [2.677, 0.127, 0.006, 0.331, 0.755, 3.271, 4.585, 3.037, 0.02, 3.682]
	[7] NLL: 19.2616 Params: [2.958, 0.95, 0.381, 0.446

KeyboardInterrupt: 

In [10]:
import model_fit
from time import time
random.seed(10)
model_fit.initialize_thread_pool(1, manual_seed = 10)
model_fitter =  model_fit.ModelFitter(model_fit.DefaultModel(), 
                           random_sample = random_sample, 
                           verbose = verbose, 
                           threads = threads)

tick = time()
q = model_fitter.fit_model(fold_data[2])
print(time() - tick)
q


Setting manual seed 10 for single-thread
Thread 0: Base Seed 10, Seed: 10, Random Number: 601088376405717203

[Preprocessing] Initial log-likelihood estimation


100%|██████████| 6/6 [00:00<00:00, 4821.96it/s]


[0] NLL: 13.4516 Params: [2.001, 0.02, 0.2, 0.05, 1.2, 0.801, 1.001, 0.4, 3.501, 5.0]
Beginning optimization of a STOCHASTIC objective function

 Iteration    f-count      E[f(x)]        SD[f(x)]           MeshScale          Method              Actions
     0           1         13.4516             nan               1                                  
[1] NLL: 19.5374 Params: [1.053, 0.68, 0.147, 0.292, 0.539, 0.952, -1.899, -1.372, -4.072, -3.799]
[2] NLL: 17.1197 Params: [1.334, 0.294, 0.414, 0.005, 1.939, -4.663, -4.106, 4.424, -3.281, 2.393]
[3] NLL: 19.682 Params: [1.632, 0.782, 0.037, 0.156, 1.082, -3.052, 3.237, 1.782, -0.542, -3.545]
[4] NLL: 19.5025 Params: [1.913, 0.396, 0.272, 0.423, 1.346, 1.294, 0.444, -4.99, -2.114, 0.264]
[5] NLL: 12.3731 Params: [2.396, 0.624, 0.304, 0.122, 1.769, -2.373, 1.753, -1.235, 2.314, -0.088]
[6] NLL: 19.1623 Params: [2.677, 0.127, 0.006, 0.331, 0.755, 3.271, 4.585, 3.037, 0.02, 3.682]
[7] NLL: 19.2616 Params: [2.958, 0.95, 0.381, 0.446, 1.61, 

KeyboardInterrupt: 

In [ ]:
from new_fit import *
random.seed(10)
initialize_thread_pool(1, manual_seed = 10)
model_fitter.fit_model(fold_data[fold_number - 1])

In [ ]:
from utils import *
fold_dataloader = DataLoader(splits[fold_number - 1])


for x, y in fold_dataloader: 
    print(x, y)

In [ ]:
move_tasks.items()

In [ ]:
list(filter(lambda x: not x[1].is_done(), move_tasks.items()))

In [42]:
import copy
import random

cutoff = 3.5
trials = fold_data[fold_number - 1]
expt_factor = 1.0

success_threshold = 1 # when "success_count" reaches this value, the tracker is considered done


n_trials = len(trials)
cumulative_cutoff = n_trials * cutoff

# trackers keeps track of the attempts and successes for IBS
data = dict(zip(trials, [{"attempt_count": 0, "success_count": 0, "log_likelihood": 0}] * n_trials))
dataloader = UltraDict(data, full_dump_size=n_trials*1024*1024, shared_lock = True) # buffer size is set to n_trials megabytes

global Lexpt
Lexpt.value =  n_trials * expt_factor


In [ ]:
scaling_factor = expt_factor / success_threshold

for _ in range(1):

    # incomplete_trials keeps track of all the trials that haven't yet reached the threshold to be considered "done"
    incomplete_trials = list(filter(lambda x: x[1]["success_count"] < success_threshold, trackers.items()))

    # if there are no more incomplete trials or if the Lexpt value is greater than the cutoff, we're done
    if not incomplete_trials or Lexpt.value > cutoff: 
        break

    # choose a trial at random, and get its tracker
    data, tracker = copy.deepcopy(random.choice(incomplete_trials))

    # accumulates all the changes we need to make to the expected log likelihood
    batch_delta_expected_log_likelihood = 0

    while tracker["success_count"] < success_threshold: 
        predicted_move = model.predict(data.board)
        actual_move = data.move.board_position

        # increment the attempt count
        tracker["attempt_count"] += 1

        # calculate the change in expected log likelihood
        # according to IBS, L = sum_k 1/k
        # where k here is the attempt count
        delta_expected_log_likelihood = -scaling_factor * 1/tracker["attempt_count"]

        # accumulate this change to the batch
        batch_delta_expected_log_likelihood += delta_expected_log_likelihood

        # update the log likelihood for this trial
        tracker["log_likelihood"] += delta_expected_log_likelihood

        # register a hit! and we're done with this trial
        if predicted_move == actual_move:
            # increment the success count and reset the attempts to 0
            tracker["success_count"] += 1
            tracker["attempt_count"] = 0
            with trackers.lock: 
                if tracker["success_count"] == trackers[data]["success_count"]:
                    trackers[data] = tracker
                    Lexpt.value += batch_delta_expected_log_likelihood
            break
        
        # if the expected log likelihood is greater than the cutoff, we're done
        if Lexpt.value + batch_delta_expected_log_likelihood > cutoff: 
            with trackers.lock: 
                Lexpt.value += batch_delta_expected_log_likelihood
            break

        




        
        delta_expected_log_likelihood = -scaling_factor * 1/tracker["attempt_count"]
        tracker["log_likelihood"] += delta_expected_log_likelihood
    
        


        



        # delta_expected_log_likelihood



